In [123]:
# Tensorflow to open-source’owy framework stworzony przez Google’a do obliczeń numerycznych. 
# Oferuje on zestaw narzędzi służących do projektowania, trenowania oraz douczania (ang. fine-tuning) sieci neuronowych.
# TensorBoard to narzędzie służące do wizualizacji danych. Pozwala prezentować w czytelny sposób m.in. grafy obliczeniowe 
# sieci neuronowych (ang. computational graph), dane wejściowe, dane wyjściowe czy postęp treningu. Dzięki temu ułatwia 
# ono znajdowanie błędów w architekturze oraz optymalizację sieci neuronowych.

In [ ]:
# Software requirements
# NVIDIA® GPU drivers —CUDA 10.1 requires 418.x or higher.
# CUDA® Toolkit —TensorFlow supports CUDA 10.1 (TensorFlow >= 2.1.0)
# CUPTI ships with the CUDA Toolkit.
# cuDNN SDK (>= 7.6)
#(Optional) TensorRT 6.0 to improve latency and throughput for inference on some models.

In [ ]:
# Source: Python for Data Science and Machine Learning Bootcamp, kurs z platformy Udemy, Stworzony przez Jose Portilla
# Ostatnia aktualizacja: 5/2020

In [124]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [125]:
os.chdir(r"C:\Users\Mateusz\Desktop\WSBProjekt")

In [128]:
conn = sqlite3.connect('data\predictions.db')
c = conn.cursor()
indicator="SP500"
# SELECT QUERY dla indexu SP500
df = pd.read_sql_query("""select x."Date", co."Index" as Crude_Oil, 
cr."Index" as Copper, s."Index" as Silver, p."Index" as Platinum,
ip."Index" as Industrial_Production, pa."Index" as Palladium, x."Index" as """+indicator+"""
 from """+indicator+""" as x
  LEFT OUTER JOIN Crude_Oil as co ON x.Date=co.Date
  LEFT OUTER JOIN Copper as cr ON x.Date=cr.Date
  LEFT OUTER JOIN Silver as s ON x.Date=s.Date
  LEFT OUTER JOIN Platinum as p ON x.Date=p.Date
  LEFT OUTER JOIN Industrial_Prod as ip ON strftime('%Y-%m', x.Date)=strftime('%Y-%m', ip.Date)
  LEFT OUTER JOIN Palladium as pa ON x.Date=pa.Date
Where x."Index" and co."Index" and cr."Index" and s."Index" and p."Index"
and ip."Index" and pa."Index" IS NOT NULL
and x.Date < '2020-04-01'
""", conn)

In [129]:
df

,Date,Crude_Oil,Copper,Silver,Platinum,Industrial_Production,Palladium,SP500
0,2010-01-04 00:00:00,81.52,3.4060,17.170,1523.90,91.685,421.40,1132.99
1,2010-01-06 00:00:00,83.12,3.4945,17.890,1558.40,91.685,427.20,1137.14
2,2010-01-07 00:00:00,82.60,3.4270,18.090,1559.40,91.685,424.55,1141.69
3,2010-01-08 00:00:00,82.74,3.4005,18.120,1570.60,91.685,425.15,1144.98
4,2010-01-11 00:00:00,82.54,3.4410,18.840,1592.50,91.685,431.95,1146.98
...,...,...,...,...,...,...,...,...
2510,2020-03-25 00:00:00,24.49,2.2040,13.965,745.50,103.664,2247.60,2475.56
2511,2020-03-26 00:00:00,22.60,2.1780,14.415,737.10,103.664,2226.10,2630.07
2512,2020-03-27 00:00:00,21.51,2.1720,14.315,740.82,103.664,2196.80,2541.47
2513,2020-03-30 00:00:00,20.09,2.1555,14.055,723.84,103.664,2197.60,2626.65


In [130]:
# Ze względu na to że dane trainowe mają mieć wartości float, usuwam kolumne daty
df = df.drop('Date',axis=1)

In [131]:
df

,Crude_Oil,Copper,Silver,Platinum,Industrial_Production,Palladium,SP500
0,81.52,3.4060,17.170,1523.90,91.685,421.40,1132.99
1,83.12,3.4945,17.890,1558.40,91.685,427.20,1137.14
2,82.60,3.4270,18.090,1559.40,91.685,424.55,1141.69
3,82.74,3.4005,18.120,1570.60,91.685,425.15,1144.98
4,82.54,3.4410,18.840,1592.50,91.685,431.95,1146.98
...,...,...,...,...,...,...,...
2510,24.49,2.2040,13.965,745.50,103.664,2247.60,2475.56
2511,22.60,2.1780,14.415,737.10,103.664,2226.10,2630.07
2512,21.51,2.1720,14.315,740.82,103.664,2196.80,2541.47
2513,20.09,2.1555,14.055,723.84,103.664,2197.60,2626.65


In [132]:
# Training And Test Data
X = df.drop('SP500',axis=1).values
y = df['SP500'].values

In [133]:
y

array([1132.99, 1137.14, 1141.69, ..., 2541.47, 2626.65, 2584.59])

In [134]:
X

array([[8.15200e+01, 3.40600e+00, 1.71700e+01, 1.52390e+03, 9.16850e+01,
        4.21400e+02],
       [8.31200e+01, 3.49450e+00, 1.78900e+01, 1.55840e+03, 9.16850e+01,
        4.27200e+02],
       [8.26000e+01, 3.42700e+00, 1.80900e+01, 1.55940e+03, 9.16850e+01,
        4.24550e+02],
       ...,
       [2.15100e+01, 2.17200e+00, 1.43150e+01, 7.40820e+02, 1.03664e+02,
        2.19680e+03],
       [2.00900e+01, 2.15550e+00, 1.40550e+01, 7.23840e+02, 1.03664e+02,
        2.19760e+03],
       [2.04800e+01, 2.22800e+00, 1.39300e+01, 7.28800e+02, 1.03664e+02,
        2.30480e+03]])

In [135]:
from sklearn.model_selection import train_test_split

In [136]:
# Dzielenie tablic na losowe podzbiory trainowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)

In [137]:
# Scalling data
from sklearn.preprocessing import MinMaxScaler

In [138]:
scaler = MinMaxScaler()

In [139]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [140]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [141]:
# Creating model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

In [142]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [143]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [144]:
from datetime import datetime

In [145]:
datetime.now().strftime("%Y-%m-%d--%H%M")

'2020-05-06--1810'

In [121]:
# WINDOWS: Use "logs\\fit"
# MACOS/LINUX: Use "logs\fit"

log_directory = 'logs\\fit'

board = TensorBoard(log_dir=log_directory,histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1)

In [122]:
# Teraz utwórzę warstwy modelu
model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=15,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [117]:
# Train the model
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop,board]
          )

Train on 1886 samples, validate on 629 samples
Epoch 1/600
1886/1886 [==============================] - 1s 419us/sample - loss: -4708.3915 - val_loss: -8285.0848
Epoch 2/600
1886/1886 [==============================] - 0s 118us/sample - loss: -15904.4233 - val_loss: -26136.2878
Epoch 3/600
1886/1886 [==============================] - 0s 210us/sample - loss: -44273.8878 - val_loss: -67926.8922
Epoch 4/600
1886/1886 [==============================] - 0s 143us/sample - loss: -105463.7195 - val_loss: -150515.0403
Epoch 5/600
1886/1886 [==============================] - 0s 113us/sample - loss: -213539.8454 - val_loss: -288960.6156
Epoch 6/600
1886/1886 [==============================] - 0s 126us/sample - loss: -391970.8112 - val_loss: -515607.6834
Epoch 7/600
1886/1886 [==============================] - 0s 118us/sample - loss: -686673.5766 - val_loss: -852242.0722
Epoch 8/600
1886/1886 [==============================] - 0s 107us/sample - loss: -1078661.8034 - val_loss: -1301462.8041
Epoch 9

1886/1886 [==============================] - 0s 172us/sample - loss: -2894180721.2047 - val_loss: -2839538751.4913
Epoch 131/600
1886/1886 [==============================] - 0s 126us/sample - loss: -2867318494.6087 - val_loss: -2893071294.0668
Epoch 132/600
1886/1886 [==============================] - 0s 146us/sample - loss: -2972110692.7169 - val_loss: -2947697876.0445
Epoch 133/600
1886/1886 [==============================] - 0s 188us/sample - loss: -2946234884.0721 - val_loss: -3002093408.0509
Epoch 134/600
1886/1886 [==============================] - 0s 103us/sample - loss: -3051746833.1029 - val_loss: -3057399698.1113
Epoch 135/600
1886/1886 [==============================] - 0s 110us/sample - loss: -3112060437.9894 - val_loss: -3113763288.9285
Epoch 136/600
1886/1886 [==============================] - 0s 114us/sample - loss: -3177886209.3574 - val_loss: -3170944061.8633
Epoch 137/600
1886/1886 [==============================] - 0s 109us/sample - loss: -3219033899.7073 - val_loss:

Epoch 256/600
1886/1886 [==============================] - 0s 106us/sample - loss: -15282046266.9099 - val_loss: -15353734279.1224
Epoch 257/600
1886/1886 [==============================] - 0s 106us/sample - loss: -15615050833.4422 - val_loss: -15506051825.7552
Epoch 258/600
1886/1886 [==============================] - 0s 107us/sample - loss: -15976123916.4878 - val_loss: -15660935314.5183
Epoch 259/600
1886/1886 [==============================] - 0s 113us/sample - loss: -15771971352.7041 - val_loss: -15813724654.9062
Epoch 260/600
1886/1886 [==============================] - 0s 100us/sample - loss: -16070474812.8102 - val_loss: -15968070517.6216
Epoch 261/600
1886/1886 [==============================] - 0s 101us/sample - loss: -16297567689.1622 - val_loss: -16124722534.1558
Epoch 262/600
1886/1886 [==============================] - 0s 108us/sample - loss: -16533577523.8515 - val_loss: -16282587469.7361
Epoch 263/600
1886/1886 [==============================] - 0s 101us/sample - loss: 

1886/1886 [==============================] - 0s 109us/sample - loss: -42633124754.3245 - val_loss: -42313975440.0763
Epoch 381/600
1886/1886 [==============================] - 0s 107us/sample - loss: -43481785201.7476 - val_loss: -42605283611.2687
Epoch 382/600
1886/1886 [==============================] - 0s 106us/sample - loss: -43053587341.9809 - val_loss: -42895467511.8601
Epoch 383/600
1886/1886 [==============================] - 0s 110us/sample - loss: -43513167828.5642 - val_loss: -43188317954.0350
Epoch 384/600
1886/1886 [==============================] - 0s 104us/sample - loss: -44776306829.1665 - val_loss: -43485750433.1701
Epoch 385/600
1886/1886 [==============================] - 0s 101us/sample - loss: -44412371304.5175 - val_loss: -43782978522.5564
Epoch 386/600
1886/1886 [==============================] - 0s 108us/sample - loss: -44554480708.4115 - val_loss: -44078761121.1701
Epoch 387/600
1886/1886 [==============================] - 0s 180us/sample - loss: -45171353788.9

1886/1886 [==============================] - 0s 116us/sample - loss: -89089438021.7688 - val_loss: -88821995953.0429
Epoch 505/600
1886/1886 [==============================] - 0s 110us/sample - loss: -90209199209.3319 - val_loss: -89289148829.5072
Epoch 506/600
1886/1886 [==============================] - 0s 247us/sample - loss: -90628534693.3277 - val_loss: -89756290883.1542
Epoch 507/600
1886/1886 [==============================] - 0s 204us/sample - loss: -90459838496.5769 - val_loss: -90223734933.7742
Epoch 508/600
1886/1886 [==============================] - 0s 167us/sample - loss: -92020146071.7540 - val_loss: -90696212414.8808
Epoch 509/600
1886/1886 [==============================] - 0s 97us/sample - loss: -92072594895.6776 - val_loss: -91169078410.3784
Epoch 510/600
1886/1886 [==============================] - 0s 94us/sample - loss: -94013356863.7964 - val_loss: -91648295654.3593
Epoch 511/600
1886/1886 [==============================] - 0s 112us/sample - loss: -92511129543.533

In [146]:
print(log_directory)

logs\fit


In [150]:
pwd

'C:\\Users\\Mateusz\\Desktop\\WSBProjekt'

### Użyj cd w wierszu poleceń, aby zmienić katalog na ścieżkę pliku zgłoszoną przez pwd lub bieżącą lokalizację pliku .py
### Następnie uruchom ten kod w wierszu polecenia lub terminalu, wpisując - 'tensorboard --logdir logs\fit'
### Tensorboard będzie działał lokalnie w twojej przeglądarce pod adresem [http://localhost:6006/](http://localhost:6006/)


In [151]:
# Możemy użyć interfejsów API tensorboard.notebook, aby uzyskać nieco większą kontrolę:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir logs\fit (started 0:24:49 ago; pid 19220)
